# Here we demonstrate how to do the moments method

The grid method is easier, so do that tutorial before this one. If you provided the correctly formatted data so that the grid tutorial works, then this should just work out of the box.


In [1]:
import sys
import pandas as pd

#you will need to change the basedir to match the location on your machine
basedir='/Users/bono/Desktop/gm2FieldAnalysis/MuonConvolution/'

# The Beam 
## load beam libs

In [2]:
path= basedir + 'tracker_info'
sys.path.append(path)
from beam_moments import get_df_tracker
from beam_moments import get_normalized_distribution_moments as gndm

## load beam data

note that the raw tracker data is lightweight and is thus included in the git repo, thus no extra steps are needed. just run the below commands.

(To understand how the beam data is structured and processed, see the get_df_tracker function in ~/tracker_info/beam_moments.py)

In [3]:
file = path + '/beamSpot.txt'
df_tracker = get_df_tracker(file=file)
mask12 = (df_tracker["Station"] == 'station12')
df_12 = df_tracker[mask12].copy()
df_12.head()

,index,radial,vertical,counts,Station
0,2,-59.0,-59.0,0.0,station12
1,3,-59.0,-57.0,0.0,station12
2,4,-59.0,-55.0,0.0,station12
3,5,-59.0,-53.0,0.0,station12
4,6,-59.0,-51.0,0.0,station12


## get the normalized moments beam moments

note that I_norm and J_norm are normal and skew beam-moments, respectively.

To understand how to decompose the beam into moments, see the get_normalized_distribution_moments function in the beam_moments modual. 

In [4]:
x = df_12['radial']
y = df_12['vertical']
weights = df_12['counts']
N = 5
I_norm, J_norm = gndm(N,x,y,weights)
print(I_norm)
print(J_norm)

[1.         0.15097058 0.10640059 0.00660348 0.00450191]
[ 0.00000000e+00 -1.28906937e-04  5.62725260e-04  4.27693398e-05
 -8.03948742e-05]


# The field

## load moments method libs

In [5]:
path = basedir + 'field_info'
sys.path.insert(0,path)
from format_field import field_team_to_standard_moments as ftts

## load moments method data

In [6]:
path = basedir + 'field_info/data/run1/all_multipoles/run1_v00/'
path = path + '60Hr_vals_uncertainties_3956-3997.pkl'
df = pd.read_pickle(path)
df.head()

DateTime        D        eD        NQ       eNQ  \
run   subrun                                                              
15921 0      2018-04-22 13:14:01  838.099  0.065085 -0.079133  0.057698   
      0      2018-04-22 13:14:03  838.078  0.065085 -0.087916  0.057698   
      0      2018-04-22 13:14:05  838.090  0.065086 -0.088702  0.057699   
      0      2018-04-22 13:14:07  838.107  0.065086 -0.099699  0.057699   
      0      2018-04-22 13:14:09  838.111  0.065086 -0.102604  0.057699   

                    SQ       eSQ       NS       eNS        SS  ...        SO  \
run   subrun                                                   ...             
15921 0       0.182330  0.056887 -1.17145  0.058112  0.303779  ...  0.266763   
      0       0.181335  0.056887 -1.17794  0.058112  0.303780  ...  0.266763   
      0       0.179374  0.056887 -1.17500  0.058112  0.303781  ...  0.266763   
      0       0.180806  0.056888 -1.17138  0.058112  0.303781  ...  0.266763   
      0       0.181792  0.056888 -1.16972  0.058112  0.303782  ...  0.266763   

                   eSO        ND      eND       SD       eSD  ctags  \
run   subrun                                                          
15921 0       0.043889  0.069171  0.04386 -1.39197  0.043907      0   
      0       0.043889  0.069171  0.04386 -1.39197  0.043907      0   
      0       0.043889  0.069171  0.04386 -1.39197  0.043907      0   
      0       0.043889  0.069171  0.04386 -1.39197  0.043907      0   
      0       0.043889  0.069171  0.04386 -1.39197  0.043907      0   

                      start_time            end_time  poor_ctags  
run   subrun                                                      
15921 0      2018-04-22 13:14:00 2018-04-22 13:14:34       231.5  
      0      2018-04-22 13:14:00 2018-04-22 13:14:34       231.5  
      0      2018-04-22 13:14:00 2018-04-22 13:14:34       231.5  
      0      2018-04-22 13:14:00 2018-04-22 13:14:34       231.5  
      0      2018-04-22 13:14:00 2018-04-22 13:14:34       231.5  

[5 rows x 23 columns]

## Calculate field quantities
Calculate the ctag averaged value of all the multipoles

In [7]:
total_ctags = df["ctags"].sum()
b = [
    (df["D"]*df["ctags"]).sum()/total_ctags,
    (df["NQ"]*df["ctags"]).sum()/total_ctags,
    (df["SQ"]*df["ctags"]).sum()/total_ctags,
    (df["NS"]*df["ctags"]).sum()/total_ctags,
    (df["SS"]*df["ctags"]).sum()/total_ctags,
    (df["NO"]*df["ctags"]).sum()/total_ctags,
    (df["NO"]*df["ctags"]).sum()/total_ctags,
    (df["ND"]*df["ctags"]).sum()/total_ctags,
    (df["ND"]*df["ctags"]).sum()/total_ctags]


Seperate normal (c) and skew (s) moments, as is done in common notation

In [8]:
c,s = ftts(b)
print(c)
print(s)

[ 8.37764260e+02 -1.00577080e-01 -1.20467157e+00  8.03280584e-03
  6.91568658e-02]
[0.         0.25299367 0.33800526 0.00803281 0.06915687]


# Finally, calculate < B >

## Load the synthesis lib

In [9]:
path = basedir + 'synthesis'
sys.path.insert(0,path)
from spacial_tools import moments_method as mm

In [10]:
moments_result = mm(c,s,I_norm,J_norm)
moments_result

837.6214149088376

Do the grid and moments methods agree?

To see how error is treated, see `~/analysis/run1_pipeline/analyze_all_runs.ipynb`